In [25]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
from sklearn.cluster import KMeans
import numpy as np
import shutil

In [26]:
# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_embedding(frame_path):
    image = Image.open(frame_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
        # Normalize embeddings
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    return image_features

In [30]:
frames_dir = "sampled_frames/"
frame_embeddings = []

for frame_file in sorted(os.listdir(frames_dir)):
    frame_path = os.path.join(frames_dir, frame_file)
    embedding = extract_embedding(frame_path)
    frame_embeddings.append(embedding)

# Stack all embeddings into a single tensor
frame_embeddings = torch.stack(frame_embeddings).squeeze()
print(f"Shape of embeddings: {frame_embeddings.shape}")  # (30, 512)


Shape of embeddings: torch.Size([30, 512])


In [28]:
# Paths
frames_dir = "frames/"  # Directory containing all frames
output_dir = "unique_frames/"  # Directory to save unique frames
os.makedirs(output_dir, exist_ok=True)

# Assuming `frame_embeddings` is your tensor of shape (30, 512)
# Step 1: Perform clustering
# Cluster the frames into key scenes (e.g., 5 scenes)
kmeans = KMeans(n_clusters=5, random_state=0).fit(frame_embeddings)
key_frame_indices = [frame_embeddings[kmeans.labels_ == i].mean(axis=0).argmax() for i in range(5)]

print(f"Key frames: {key_frame_indices}")


Key frames: [tensor(376), tensor(376), tensor(376), tensor(389), tensor(39)]


In [ ]:
output_dir = "unique_frames/"  # Directory to save unique frames
os.makedirs(output_dir, exist_ok=True)

# Assuming `frame_embeddings` is your tensor of shape (30, 512)
# Step 1: Perform clustering
n_clusters = 8  # Number of unique groups/scenes
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(frame_embeddings)

# Step 2: Select one representative frame per cluster
unique_frames = []

for cluster_id in range(n_clusters):
    # Get indices of frames in the current cluster
    cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
    
    # Find the frame closest to the cluster centroid
    cluster_center = kmeans.cluster_centers_[cluster_id]
    distances = np.linalg.norm(frame_embeddings[cluster_indices] - cluster_center, axis=1)
    closest_frame_idx = cluster_indices[distances.argmin()]
    
    unique_frames.append(closest_frame_idx)

# Debugging step: Check the length of `frame_files` and `unique_frames`
frame_files = sorted(os.listdir(frames_dir))  # Ensure sorted order of frames

# Debugging step: Check the length of `frame_files` and `unique_frames`
print(f"Total frames in directory: {len(frame_files)}")
print(f"Number of unique frames selected: {len(unique_frames)}")
i = 0
# Step 3: Save the selected unique frames as images
for idx in unique_frames:
    # Ensure the index is within bounds
    if idx < len(frame_files):
        frame_path = os.path.join(frames_dir, frame_files[idx])
        
        # Open the image using PIL
        img = Image.open(frame_path)
        
        # Define save path
        save_path = os.path.join(output_dir, f"unique_frame_{i}.png")
        i += 1
        # Save the image as a PNG
        img.save(save_path)
        print(f"Saved: {frame_files[idx]} as {save_path}")
    else:
        print(f"Error: Index {idx} is out of range for frame files.")

print(f"Saved {len(unique_frames)} unique frames as images to {output_dir}.")


Total frames in directory: 30
Number of unique frames selected: 8
Saved: frame_012.jpg as unique_frames/unique_frame_12.png
Saved: frame_029.jpg as unique_frames/unique_frame_29.png
Saved: frame_003.jpg as unique_frames/unique_frame_3.png
Saved: frame_020.jpg as unique_frames/unique_frame_20.png
Saved: frame_005.jpg as unique_frames/unique_frame_5.png
Saved: frame_014.jpg as unique_frames/unique_frame_14.png
Saved: frame_001.jpg as unique_frames/unique_frame_1.png
Saved: frame_024.jpg as unique_frames/unique_frame_24.png
Saved 8 unique frames as images to unique_frames/.
